In [1]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from ens_util import *
from math import isnan
from sklearn.calibration import calibration_curve
plt.rcParams['figure.figsize'] = [9,9]

In [2]:
data_files = glob('data_crop*.nc')
res_files = glob('2014*Z_crs_gauss.nc')
coordx = [240,258,266,267,271,271,271,272]
coordy = [78,42,54,59,49,50,51,49]
for x in range(248,266):
    for y in range(68,81):
        coordx.append(x)
        coordy.append(y)
weights = [59,49.87,13,2.5,2.04,13.7,5,8]
for i in range(len(coordx)-8):
    weights.append(0.5)
weights = np.array(weights)

thresh = 0.2
data = build_datadf(data_files,coordx,coordy)
(fr15,fr30,fr45,fr60) = build_solar_fractions_weightprob(res_files,data.index,weights,data.columns,coordx,coordy,thresh)

KeyboardInterrupt: 

In [ ]:
data_sol = pd.DataFrame(index=data.index,columns=[0])
for t in data.index:
    weighted = np.multiply(data.loc[t].values,weights)
    avg = np.sum(weighted)/np.sum(weights)
    if(avg<0): avg=0
    data_sol.at[t]=avg
fr = [fr15,fr30,fr45,fr60]
data = data_sol

In [ ]:
index_dates = data.index.date
index_dates_unique = np.unique(index_dates)
index_times = data.index
bs15 = pd.DataFrame(index=index_dates_unique,columns=data.columns)
bs30 = pd.DataFrame(index=index_dates_unique,columns=data.columns)
bs45 = pd.DataFrame(index=index_dates_unique,columns=data.columns)
bs60 = pd.DataFrame(index=index_dates_unique,columns=data.columns)
bs = [bs15,bs30,bs45,bs60]

data_bin = data.mask(data<thresh,other=1)
data_bin = data_bin.mask(data_bin!=1.0,other=0)

for c in data.columns:
    for d in index_dates_unique:
        times = index_times[np.where(index_dates==d)]
        sat = data_bin.loc[times,[c]]
        for f in range(4):
            frac = fr[f].loc[times,[c]]
            bs_vals = ((frac-sat)**2).values[:,0].astype(np.float)
            valid = np.where(np.invert(np.isnan(bs_vals)))
            bs[f].at[d,c] = np.sum(bs_vals[valid])/len(bs_vals[valid])
            
dec15 = pd.DataFrame(index=index_dates_unique,columns=['rel','res','sig','bs'])
dec30 = pd.DataFrame(index=index_dates_unique,columns=['rel','res','sig','bs'])
dec45 = pd.DataFrame(index=index_dates_unique,columns=['rel','res','sig','bs'])
dec60 = pd.DataFrame(index=index_dates_unique,columns=['rel','res','sig','bs'])
dec = [dec15,dec30,dec45,dec60]

c=0
for d in index_dates_unique:
    times = index_times[np.where(index_dates==d)]
    sat = data_bin.loc[times,[c]]
    for f in range(4):
        frac = fr[f].loc[times,[c]]
        bs_vals = ((frac-sat)**2).values[:,0].astype(np.float)
        sat_vals = sat.values[:,0].astype(np.float)
        frac_vals = frac.values[:,0].astype(np.float)
        valid = np.where(np.invert(np.isnan(bs_vals)))
        bs_valid = bs_vals[valid]
        sat_valid = sat_vals[valid]
        frac_valid = frac_vals[valid]
        
        unique,counts = np.unique(frac_valid,return_counts=True)
        n = np.sum(counts)
        obar_i = np.zeros(len(unique))
        for j in range(len(unique)):
            y = unique[j]
            wh = np.where(frac_valid==y)
            obar_i[j] = np.sum(sat_valid[wh])/counts[j]
        obar = np.sum(sat_valid)/n

        sig = obar*(1-obar)
        rel = np.sum(np.multiply(counts,(unique-obar_i)**2))/n
        res = np.sum(np.multiply(counts,(obar_i-obar)**2))/n
        dec[f].at[d,'rel'] = rel
        dec[f].at[d,'res'] = res
        dec[f].at[d,'sig'] = sig
        dec[f].at[d,'bs'] = rel-res+sig


In [ ]:
avg_ci = [0.07326633937050787, 0.20778712302975885, 0.11192718638706237, 0.06896607381887883, 0.14332269534242617, 0.25472343970477335, 0.04777688933358877, 0.8596485152665307, 0.19564074560864855, 0.06170793115020668, 0.021583633585999476, 0.03259704694624734, 0.03989605709146575, 0.5405269801579695, 0.013546266323385918, 0.06124534490048805, 0.05463109817092926, 0.03606780100144922, 0.0638700738320475, 0.03620355300317931, 0.04244338007354623, 0.009881199632866159, 0.033942529421483604, 0.009342719017178461, 0.05944429052055939, 0.10105394353402507, 0.6654091999523657, 0.0021613105745977136, 0.013400553711767014, 0.017006162006099224, 0.09350540001254598, 0.10001397910002892, 0.006636670686108557, 0.09580385996389294, 0.08552496468202024, 0.16219456934530282, 0.10637936657699525, 0.017298070474985355, 0.0015729371576519585]

In [ ]:
plt.rcParams['figure.figsize'] = [12,12]
ymin= 0
ymax = 0.3
rows = 1; cols = 1;

x = list(dec15.index)
xpos = np.array(range(len(x)))+1
width = 0.15
dx = width/2

fig,ax = plt.subplots(rows,cols)
ax.set_ylim([ymin,ymax])
ax.set_xlim([0,len(xpos)+1])
ax.set_xticks(xpos)
ax.set_xticklabels(x,rotation='vertical')

ax.bar(xpos-3*dx,dec15.values[:,0],width=width,color='b',label='reliability')
ax.bar(xpos-dx,dec15.values[:,1],width=width,color='g',label='resolution')
ax.bar(xpos+dx,dec15.values[:,2],width=width,color='r',label='uncertainty')
ax.bar(xpos+3*dx,dec15.values[:,3],width=width,color='k',label='brier score')

for i in range(1,len(x)+1):
    alpha = 0.5*avg_ci[i-1]
    ax.fill_between(np.arange(i-0.5,i+0.501,0.1),0,0.3,facecolor='k',alpha=alpha)

ax.set_title("15 minutes")

#fig.autofmt_xdate()
fig.tight_layout()
fig.suptitle("Brier Score Timeseries",fontsize=16)
fig.subplots_adjust(top=0.92)
plt.legend()
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [12,18]
ymin= 0
ymax = 0.4
rows = 4; cols = 1;
fig,ax = plt.subplots(rows,cols,sharex=True)
for r in range(rows):
    for c in range(cols):
        ax[r].set_ylim([ymin,ymax])
        ax[r].set_xlim([0,len(xpos)+1])
        ax[r].set_xticks(xpos)
        ax[r].set_xticklabels(x,rotation='vertical')
        
ax[0].bar(xpos-3*dx,dec15.values[:,0],width=width,color='b',label='reliability')
ax[0].bar(xpos-dx,dec15.values[:,1],width=width,color='g',label='resolution')
ax[0].bar(xpos+dx,dec15.values[:,2],width=width,color='r',label='uncertainty')
ax[0].bar(xpos+3*dx,dec15.values[:,3],width=width,color='k',label='brier score')

ax[1].bar(xpos-3*dx,dec30.values[:,0],width=width,color='b',label='reliability')
ax[1].bar(xpos-dx,dec30.values[:,1],width=width,color='g',label='resolution')
ax[1].bar(xpos+dx,dec30.values[:,2],width=width,color='r',label='uncertainty')
ax[1].bar(xpos+3*dx,dec30.values[:,3],width=width,color='k',label='brier score')

ax[2].bar(xpos-3*dx,dec45.values[:,0],width=width,color='b',label='reliability')
ax[2].bar(xpos-dx,dec45.values[:,1],width=width,color='g',label='resolution')
ax[2].bar(xpos+dx,dec45.values[:,2],width=width,color='r',label='uncertainty')
ax[2].bar(xpos+3*dx,dec45.values[:,3],width=width,color='k',label='brier score')

ax[3].bar(xpos-3*dx,dec60.values[:,0],width=width,color='b',label='reliability')
ax[3].bar(xpos-dx,dec60.values[:,1],width=width,color='g',label='resolution')
ax[3].bar(xpos+dx,dec60.values[:,2],width=width,color='r',label='uncertainty')
ax[3].bar(xpos+3*dx,dec60.values[:,3],width=width,color='k',label='brier score')

for a in range(4):
    for i in range(1,len(x)+1):
        alpha = 0.5*avg_ci[i-1]
        ax[a].fill_between(np.arange(i-0.5,i+0.501,0.1),0,0.4,facecolor='k',alpha=alpha)

ax[0].set_title("15 minutes")
ax[1].set_title("30 minutes")
ax[2].set_title("45 minutes")
ax[3].set_title("60 minutes")

ax[0].legend()

#fig.autofmt_xdate()
fig.tight_layout()
fig.suptitle("Brier Score Timeseries",fontsize=16)
fig.subplots_adjust(top=0.92)
plt.show()

In [ ]:
c=0

index_dates = data.index.date
index_dates_unique = np.unique(index_dates)
index_times = data.index

data_bin = data.mask(data<thresh,other=1)
data_bin = data_bin.mask(data_bin!=1.0,other=0)

sat = data_bin.loc[:,[c]]
for f in range(4):
    frac = fr[f].loc[:,[c]]
    bs_df = ((frac-sat)**2)
    times = bs_df.index.values
    bs_vals = bs_df.values[:,0].astype(np.float)
    valid = np.where(np.invert(np.isnan(bs_vals)))
    bs_valid = bs_vals[valid]
    valid_times = times[valid]
    sat_vals = sat.loc[valid_times,[c]].values[:,0]
    obar = np.sum(sat_vals)/len(sat_vals)
    
    brier = np.sum(bs_valid)/len(bs_valid)
    print((f+1)*15,'minutes :',brier)
    #print('BS = ',brier)
    #print('obar = ',obar)

In [ ]:
coord = 0
ytrue = data_bin[0].values.astype(np.float)

yprob15 = fr15.loc[data_bin.index.values].values[:,coord].astype(np.float)
yprob30 = fr30.loc[data_bin.index.values].values[:,coord].astype(np.float)
yprob45 = fr45.loc[data_bin.index.values].values[:,coord].astype(np.float)
yprob60 = fr60.loc[data_bin.index.values].values[:,coord].astype(np.float)

valid15 = [i for i in range(len(yprob15)) if not np.isnan(yprob15[i])]
valid30 = [i for i in range(len(yprob30)) if not np.isnan(yprob30[i])]
valid45 = [i for i in range(len(yprob45)) if not np.isnan(yprob45[i])]
valid60 = [i for i in range(len(yprob60)) if not np.isnan(yprob60[i])]

ptrue15,ppred15 = calibration_curve(ytrue[valid15],yprob15[valid15])
ptrue30,ppred30 = calibration_curve(ytrue[valid30],yprob30[valid30])
ptrue45,ppred45 = calibration_curve(ytrue[valid45],yprob45[valid45])
ptrue60,ppred60 = calibration_curve(ytrue[valid60],yprob60[valid60])


In [ ]:
plt.rcParams['figure.figsize'] = [9, 9]

obar = 0.837 # similar obar for all outlook times

xmin = 0
ymin = xmin
xmax = 1
ymax = xmax
x = [xmin,xmax]
y = [ymin,ymax]
nsy = [obar/2,(1+obar)/2]
rows = 2; cols = 2;
fig,axarr = plt.subplots(rows,cols)
for r in range(rows):
    for c in range(cols):
        axarr[r,c].set_aspect('equal','box')
        axarr[r,c].axis([xmin,xmax,ymin,ymax])
        axarr[r,c].plot(x,y,c='k',lw=1)
        axarr[r,c].set_xlabel("fraction predicted")
        axarr[r,c].set_ylabel("fraction of positives")
        axarr[r,c].axhline(obar,ls='--',c='k',lw=1)
        axarr[r,c].axvline(obar,ls='--',c='k',lw=1)
        axarr[r,c].plot(x,nsy,c='k',lw=1)
        axarr[r,c].fill_between([0,obar],0,[obar/2,obar],facecolor='k',alpha=0.1)
        axarr[r,c].fill_between([obar,1],[obar,(1+obar)/2],1,facecolor='k',alpha=0.1)

n = np.array([64,11,11,37,549])
err15 = 1.96*np.sqrt(ptrue15*(1-ptrue15)/n)
err30 = 1.96*np.sqrt(ptrue30*(1-ptrue30)/n)
err45 = 1.96*np.sqrt(ptrue45*(1-ptrue45)/n)
err60 = 1.96*np.sqrt(ptrue60*(1-ptrue60)/n)

        
axarr[0,0].scatter(ppred15,ptrue15)
axarr[0,0].errorbar(ppred15,ptrue15,err15,capsize=5)
axarr[0,0].plot(ppred15,ptrue15)

axarr[0,1].scatter(ppred30,ptrue30)
axarr[0,1].errorbar(ppred30,ptrue30,err30,capsize=5)
axarr[0,1].plot(ppred30,ptrue30)

axarr[1,0].scatter(ppred45,ptrue45)
axarr[1,0].errorbar(ppred45,ptrue45,err45,capsize=5)
axarr[1,0].plot(ppred45,ptrue45)

axarr[1,1].scatter(ppred60,ptrue60)
axarr[1,1].errorbar(ppred60,ptrue60,err60,capsize=5)
axarr[1,1].plot(ppred60,ptrue60)


axarr[0,0].set_title("15 minutes")
axarr[0,1].set_title("30 minutes")
axarr[1,0].set_title("45 minutes")
axarr[1,1].set_title("60 minutes")

fig.tight_layout()
fig.suptitle("Reliability Curves",fontsize=16)
fig.subplots_adjust(top=0.92)
plt.show()

In [ ]:
n = plt.hist(yprob15[valid15],bins=5)
plt.ylim([0,600])
plt.show()
print(n[0])

Correction: Do the threshold after the weighting.  Flip steps: weighting the cloud index before threshold instead of weighting probabilities after weighting.  Add in an average over a grid area for the house PV. For weighting, weight houses so that total output mirrors actual total residential output.
Turn off Gaussian weighting for the grid.  Add upper left points for plants outside grid.  For overlapping 7x7 regions, collapse into single points and add output. Include first two plants, put at x=240.

Add in residential grid.  Come up with P(x,y,cloudy/not) equations that describe the code.  Try doing the probability weighting rather than the CI weighting as I had done originally, but across all solar plant locations.